<a href="https://colab.research.google.com/github/vernonconnelldavies/20025299_DataAnalytics/blob/main/assignment_linear_regression_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial we will train and test a linear regressor. This is something you have already done in part 3 of the course document. So, I will miss out all of the extra notes and just get to business.

In [35]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/vernonconnelldavies/20025299_DataAnalytics/refs/heads/main/final_pre_covid_data.csv', index_col=0, )

In [36]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

     day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  mxpsd  \
185    4  2013   1   1     2013-01-01  37.8  23.6  1011.9   10.0   6.1    8.9   
186    5  2013   1   2     2013-01-02  27.1  10.5  1016.8   10.0   5.3    9.9   
187    6  2013   1   3     2013-01-03  28.4  14.1  1020.6   10.0   3.7    8.0   
188    7  2013   1   4     2013-01-04  33.4  18.6  1017.0   10.0   6.5   13.0   
189    2  2013   1   5     2013-01-05  36.1  18.7  1020.6   10.0   6.6   12.0   
190    1  2013   1   6     2013-01-06  38.3  25.0  1019.5    8.5   5.3    8.9   

     gust   max   min  prcp   sndp  fog  NUM_COLLISIONS  
185  19.0  39.9  33.1   0.0  999.9    0             381  
186  19.0  33.1  21.9   0.0  999.9    0             480  
187  15.0  32.0  24.1   0.0  999.9    0             549  
188  24.1  37.0  30.0   0.0  999.9    0             505  
189  21.0  42.1  32.0   0.0  999.9    0             389  
190  17.1  46.0  34.0   0.0  999.9    0             393  


In [37]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_COLLISIONS = 1000

In [38]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.17.1


# Day and NUM_COLLISIONS

In [39]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration, in this case, I have gone for day, temp, wdsp
df_input_data_day = [df["day"], df["NUM_COLLISIONS"]]
# create headers for our new dataframe. These should correlate with the above.
df_input_headers_day = ["day", "NUM_COLLISIONS"]
# create a final dataframe using our new dataframe and headers.
df_input_day = pd.concat(df_input_data_day, axis=1, keys=df_input_headers_day)

In [40]:
# construct a training set for runnign through the model and a test set, we do this by using sample with 0.8 for an 80% training set and 20% for test.
training_set_day = df_input_day.sample(frac=0.8, random_state=0)
test_set_day = df_input_day.drop(training_set_day.index)

In [41]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features_day = training_set_day.copy()
test_features_day = test_set_day.copy()

training_labels_day = training_features_day.pop('NUM_COLLISIONS')
test_labels_day = test_features_day.pop('NUM_COLLISIONS')

In [42]:
# Here I have put in a scale factor and divided by it. In this dataset, I had already normalised and thus it is 1. However, 600000 is what would make sense based on the data here and we can use this later when testing our model..
training_labels_day = training_labels_day/SCALE_NUM_COLLISIONS
test_labels_day = test_labels_day/SCALE_NUM_COLLISIONS

In [43]:
print(training_features_day)

      day
787     4
748     7
1818    6
461     7
1345    1
...   ...
1648    4
1386    2
1779    1
2478    2
609     2

[2045 rows x 1 columns]


In [44]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser_day = tf.keras.layers.Normalization(input_shape=[1,], axis=None) # tf.keras.layers.Normalization(axis=-1)
normaliser_day.adapt(np.array(training_features_day))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
# I have decided to call the model, model_1. We add our normaliser and we are expecting a single output.
model_0 = tf.keras.Sequential([
    normaliser_day,
    layers.Dense(units=1)
])

In [46]:
model_0.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [47]:
# now we are going to fit the model where we require the training features and labels. We will run it 100 times i.e. epochs and we have applied a further 20% validation split.

%%time
history = model_0.fit(
    training_features_day,
    training_labels_day,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.9 s, sys: 716 ms, total: 14.6 s
Wall time: 15.3 s


In [48]:
# Now, we will evaluate our model using the test features and labels.
mean_absolute_error_model_0 = model_0.evaluate(
    test_features_day,
    test_labels_day, verbose=0)

In [49]:
# The mean absolute error of the model can be printed out. Remember, we want to minimise this. Perhaps a model with just day and NUM_TRIPS would be better. It will also vary on each training run due to randomisation.
print(mean_absolute_error_model_0)

0.07194401323795319


In [50]:
input_0 = pd.DataFrame.from_dict(data =
				{
            'day' : [1,2,3,4,5,6,7],
        })

In [51]:
# next we can check this out, you can multiply by 600000 to get more realistic NUM_TRIPS values.
linear_day_predictions_0 = model_0.predict(input_0)*SCALE_NUM_COLLISIONS # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data
print(linear_day_predictions_0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[469.0919 ]
 [503.6481 ]
 [538.20435]
 [572.76056]
 [607.3168 ]
 [641.87305]
 [676.42926]]


# Day, Temp, Mean Dew Point, NUM_COLLISIONS


In [52]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration, in this case, I have gone for day, temp, wdsp
df_input_data = [df["day"], df["temp"],df["dewp"], df["NUM_COLLISIONS"]]
# create headers for our new dataframe. These should correlate with the above.
df_input_headers = ["day", "temp", "dewp", "NUM_COLLISIONS"]
# create a final dataframe using our new dataframe and headers.
df_input = pd.concat(df_input_data, axis=1, keys=df_input_headers)

In [53]:
# construct a training set for runnign through the model and a test set, we do this by using sample with 0.8 for an 80% training set and 20% for test.
training_set = df_input.sample(frac=0.8, random_state=0)
test_set = df_input.drop(training_set.index)

In [54]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features = training_set.copy()
test_features = test_set.copy()

training_labels = training_features.pop('NUM_COLLISIONS')
test_labels = test_features.pop('NUM_COLLISIONS')

In [55]:
# Here I have put in a scale factor and divided by it. In this dataset, I had already normalised and thus it is 1. However, 600000 is what would make sense based on the data here and we can use this later when testing our model..
training_labels = training_labels/SCALE_NUM_COLLISIONS
test_labels = test_labels/SCALE_NUM_COLLISIONS

In [56]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [57]:
# I have decided to call the model, model_1. We add our normaliser and we are expecting a single output.
model_1 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

In [58]:
# more boiler plate for creating a sequential model, we need an optimiser and loss parameter. Here we are going to be using the mean absolute error MAE
model_1.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [59]:
# now we are going to fit the model where we require the training features and labels. We will run it 100 times i.e. epochs and we have applied a further 20% validation split.

%%time
history = model_1.fit(
    training_features,
    training_labels,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 13.6 s, sys: 704 ms, total: 14.3 s
Wall time: 15.3 s


In [60]:
# Now, we will evaluate our model using the test features and labels.
mean_absolute_error_model_1 = model_1.evaluate(
    test_features,
    test_labels, verbose=0)

In [61]:
# The mean absolute error of the model can be printed out. Remember, we want to minimise this. Perhaps a model with just day and Collisions would be better. It will also vary on each training run due to randomisation.
print(mean_absolute_error_model_1)

0.076545849442482


In [62]:
# we create a custom dataframe with 3 values per feature.
input_1 = pd.DataFrame.from_dict(data =
				{
            'day' : [6,2,3],
            'temp' : [48.6, 41.9, 42.8],
            'dewp':[46.2,30.2,25.2]
        })

In [63]:
input_1.head()

,day,temp,dewp
0,6,48.6,46.2
1,2,41.9,30.2
2,3,42.8,25.2


In [64]:
# next we can check this out, you can multiply by 600000 to get more realistic NUM_TRIPS values.
linear_day_predictions_1 = model_1.predict(input_1[:3])*SCALE_NUM_COLLISIONS # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data
print(linear_day_predictions_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[719.84674]
 [578.3236 ]
 [612.069  ]]


In [65]:
# same as above
input_2 = pd.DataFrame.from_dict(data =
				{
            'day' : [6,4,4],
            'temp' : [59.3, 32.6, 39.6],
            'dewp':[48.1,10.9,22.4],
        })

In [66]:
linear_day_predictions_2 = model_1.predict(input_2[:3])*SCALE_NUM_COLLISIONS # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data
print(linear_day_predictions_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[727.9814]
 [636.5103]
 [643.0429]]
